In [1]:
# PTT title generator from content base on seq2seq model.
import pandas as pd
import time
import csv
import os
import types 

#Initialize for files name and path.
base_dir = 'C:/Users/User/Raw data/PTT'
totalboard_name = 'TJ-BG-AT-SX-CC-MV'
board_name = ['Tech_job','Boy-Girl','AllTogether','sex','C_Chat','movie']
board_dict_index = {'Tech_job':0,'Boy-Girl':1,'AllTogether':2,'sex':3,'C_Chat':4,'movie':5}
max_word_length = 256
min_word_length = 100
max_title_length = 20
label_newnum = len(board_name)

new_content_dir = []
totalboard_dir = os.path.join(base_dir, totalboard_name)
if not os.path.exists(totalboard_dir):
    os.makedirs(totalboard_dir)
for name_ind in range(len(board_name)):
    # New data path
    new_content_dir.append(os.path.join(totalboard_dir, board_name[name_ind]+'_content'))
    if not os.path.exists(new_content_dir[name_ind]):
        os.makedirs(new_content_dir[name_ind])

from sklearn.cross_validation import train_test_split
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
import jieba

# Import dictionary fron jieba and PTT.
jieba.set_dictionary('C:/Users/User/Anaconda3/Lib/site-packages/jieba/dict.txt.big')
jieba.load_userdict('C:/Users/User/Anaconda3/Lib/site-packages/jieba/userdict.txt')

text_content = []
text_label = []
text_title = []

GO_word = '<GO>' #解碼器端的句子起始標識符。
EOS_word = '<EOS>' #解碼器端的句子結束標識符。
PAD_word = '<PAD>' #補全字符。
UNK_word = '<UNK>' #低頻詞或者一些未遇到過的詞等。

# Load PTT content and information.
#dfs_total = pd.read_csv(os.path.join(totalboard_dir, totalboard_name + '-test.csv'))
dfs_total = pd.read_csv(os.path.join(totalboard_dir, totalboard_name + '.csv'))
len_total = len(dfs_total)

for dfs_index in range(len_total):
    read_index = int(dfs_total.iloc[dfs_index, 0])
    type_name = str(dfs_total.iloc[dfs_index, 1])
    word_length = int(dfs_total.iloc[dfs_index, 5])
    title_name = str(dfs_total.iloc[dfs_index, 9])
    this_board_name = str(dfs_total.iloc[dfs_index, 10])
    
    # Add start and end character.
    text = GO_word + ' '
    with open(os.path.join(new_content_dir[board_dict_index[this_board_name]], str(read_index) + '.csv'),
                'r', encoding = 'utf-8-sig') as file:
        csvCursor = csv.reader(file)
        for rows in csvCursor:
            for row in rows:
                # Read content and remove empty.
                text = text + row
        else:
            text = text + ' ' + EOS_word
    
    # Split titles to words by Jieba.
    title_name.encode('utf-8-sig')
    title_jieba = jieba.cut(title_name, cut_all=False)
    title_name = GO_word
    word_length = 0
    for word in title_jieba:
        word_length += 1
        title_name = title_name + ' ' + word
    else:
        title_name = title_name + ' ' + EOS_word

    text_content.append(text)
    text_label.append(board_dict_index[this_board_name])
    text_title.append(title_name)
    file.close()

# Build vocabulary and convert content to sequence by Keras tool.
words_limit = 60000
tokenizer = Tokenizer(num_words=words_limit, 
                      filters='!"#$%&()*+,-./:;=?@[\\]^_`{|}~\t\n　，。！：；、？﹝﹞「」『』（）｛｝［］【】《》“”‘’＼｜〝〞‵′＋－＊／＝≦≧＿＠＃＄％︿＆～§◎．※ㄧ↔│○●☆★◇◆□■▽▼△▲㊣⊙⊕ˍ…﹌﹋﹎﹍﹉﹊‥–↑↓←→↖↗↙↘∥∕℅≒≡∩∪∞￣＿◤◥◣◢∵∴〒⊥∠⊿┼┴┬┤├▔─│▕┌┐└┘╭╮╰╯═╞╪╡╔╦╗╠╬╣╚╩╝╒╤╕╘╧╛╓╥╖╟╫╢╙╨╜║▓╱╲╳▁▂▄▅▆▇█▏▎▍▌▋▊▉▁▔', 
                      lower=False,
                      split=" ")
                      #oov_token=UNK_word)
tokenizer.fit_on_texts(text_content+text_title)
vocab = tokenizer.word_index
vocab_counts = tokenizer.word_counts
vocab_docs = tokenizer.word_docs

print(len(vocab))
anti_vocab = {}
vocab_keys = list(vocab.keys())
for vocab_word in vocab:
    vocab_index = vocab[vocab_word]
    anti_vocab[vocab_index] = vocab_keys[vocab_index-1]
    if vocab_index % 1000 == 0 or vocab_index < 20:
        print(vocab[vocab_word], vocab_word, anti_vocab[vocab_index], vocab_counts[vocab_word], vocab_docs[vocab_word])

    
from keras.utils import np_utils

x_train, x_test, y_train, y_test = train_test_split(text_content, text_title, test_size=0.2, random_state=9487)

# 将每个词用词典中的数值代替
x_train_word_ids = tokenizer.texts_to_sequences(x_train)
x_test_word_ids = tokenizer.texts_to_sequences(x_test)
y_train_word_ids = tokenizer.texts_to_sequences(y_train)
y_test_word_ids = tokenizer.texts_to_sequences(y_test)
# 序列模式
x_train = pad_sequences(x_train_word_ids, maxlen=max_word_length+2, padding='post', truncating='post')
x_test = pad_sequences(x_test_word_ids, maxlen=max_word_length+2, padding='post', truncating='post')
y_train = pad_sequences(y_train_word_ids, maxlen=max_title_length+2, padding='post', truncating='post')
y_test = pad_sequences(y_test_word_ids, maxlen=max_title_length+2, padding='post', truncating='post')
# 序列長度
x_train_size = [len(word_ids) for word_ids in x_train_word_ids]
x_test_size = [len(word_ids) for word_ids in x_test_word_ids]
y_train_size = [len(word_ids) for word_ids in y_train_word_ids]
y_test_size = [len(word_ids) for word_ids in y_test_word_ids]

C:\Users\User\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
Using TensorFlow backend.
Building prefix dict from C:\Users\User\Anaconda3\Lib\site-packages\jieba\dict.txt.big ...
Loading model from cache C:\Users\User\AppData\Local\Temp\jieba.ufa6ae29b0cbce8b45e006c7fa30eaaf8.cache
Loading model cost 1.000 seconds.
Prefix dict has been built succesfully.


199446
1 的 的 479528 108422
2 我 我 187622 60294
3 <GO> <GO> 172800 172800
4 <EOS> <EOS> 172800 172800
5 是 是 157302 68972
6 了 了 129931 60543
7 有 有 108041 59614
8 在 在 97066 54361
9 也 也 86587 48759
10 都 都 75404 45560
11 就 就 71400 42410
12 不 不 66238 42797
13 很 很 54725 34820
14 會 會 54210 36201
15 他 他 49560 20170
16 跟 跟 48539 31439
17 看 看 48476 30452
18 你 你 48168 21707
19 想 想 47770 34168
1000 跳 跳 1246 1022
2000 整合 整合 607 489
3000 往往 往往 366 345
4000 007 007 255 156
5000 處於 處於 192 188
6000 一般來說 一般來說 154 150
7000 何 何 125 101
8000 有鬼 有鬼 105 100
9000 備 備 89 89
10000 半年前 半年前 78 78
11000 掉下來 掉下來 68 66
12000 托爾 托爾 61 46
13000 傻子 傻子 54 51
14000 活得 活得 49 48
15000 寫給 寫給 44 41
16000 希斯 希斯 41 31
17000 毆打 毆打 37 33
18000 編制 編制 34 29
19000 激怒 激怒 32 30
20000 神秘性 神秘性 30 5
21000 睡意 睡意 27 27
22000 真情 真情 26 24
23000 寫些 寫些 24 24
24000 野蠻女友 野蠻女友 23 14
25000 20th 20th 21 17
26000 世家 世家 20 20
27000 天降 天降 19 18
28000 人卡 人卡 18 13
29000 決定權 決定權 17 17
30000 始於 始於 16 15
31000 初號機 初號機 16 10
32000 早睡早起 早睡早起 15 15
33000 記號 記號

In [52]:
import os
import time
import random
import numpy as np
import tensorflow as tf
from tensorflow.python.layers.core import Dense
from tensorflow.contrib import layers

patience = 0
num_epoch = 20
batch_size = 64
test_batch_size = 64

rnn_size = 150
num_layers = 4
slot_size = 172
embedding_size = 300
vocab_size = words_limit
limit_generation_len = max_title_length + 2
lr = 1e-3

train_graph = tf.Graph()
with train_graph.as_default():
    ################
    # model input tensor
    ################
    inputs = tf.placeholder(tf.int32, [None, None], name='inputs')
    targets = tf.placeholder(tf.int32, [None, None], name='targets')
    validation_batch_size = tf.shape(targets)[0]
    start_tokens = tf.tile(tf.constant([vocab[GO_word]], dtype=tf.int32),
                               [validation_batch_size], name='start_tokens')
    
    input_seq_len = tf.placeholder(tf.int32, (None,), name='input_seq_len')
    target_seq_len = tf.placeholder(tf.int32, (None,), name='target_seq_len')
    max_target_seq_len = tf.reduce_max(target_seq_len, name='max_target_seq_len')
    
    limit_target_seq_len = tf.placeholder(tf.int32, (), name='limit_target_seq_len')
    
    
    ################
    # Embedding for encoder and decoder
    ################
    embeddings = tf.Variable(tf.random_uniform([vocab_size, embedding_size]))
    encoder_embed_input = tf.nn.embedding_lookup(embeddings, inputs)
    decoder_embed_input = tf.nn.embedding_lookup(embeddings, targets)
    
    ################
    # Encoder by LSTM
    ################
    # LSTM cell
    def get_lstm_cell(rnn_size):
        lstm_cell = tf.contrib.rnn.LSTMCell(rnn_size, 
                                            initializer=tf.random_uniform_initializer(-0.1, 0.1)) 
        #lstm_cell.add_variable(regularizer=layers.l2_regularizer(0.1))
        return lstm_cell

    # Struct encoder by LSTM bi-directional multi-layer
    encoder_cell = tf.contrib.rnn.MultiRNNCell([get_lstm_cell(rnn_size) for _ in range(num_layers)])
    encoder_outputs, encoder_state = tf.nn.dynamic_rnn(encoder_cell, encoder_embed_input, 
                                                       sequence_length=input_seq_len, 
                                                       dtype=tf.float32)
    '''
    encoder_f_cell = tf.contrib.rnn.MultiRNNCell([get_lstm_cell(rnn_size) for _ in range(num_layers)])
    encoder_b_cell = tf.contrib.rnn.MultiRNNCell([get_lstm_cell(rnn_size) for _ in range(num_layers)])
    encoder_bi_outputs, encoder_bi_state = tf.nn.bidirectional_dynamic_rnn(encoder_f_cell, 
                                                                           encoder_b_cell, 
                                                                           encoder_embed_input, 
                                                                           sequence_length=input_seq_len, 
                                                                           dtype=tf.float32)
    encoder_outputs = tf.concat((encoder_bi_outputs[0], encoder_bi_outputs[1]), 2)
    
    encoder_state_c = tf.concat((encoder_bi_state[0].c, encoder_bi_state[1].c), 1)
    encoder_state_h = tf.concat((encoder_bi_state[0].h, encoder_bi_state[1].h), 1)
    encoder_state = tf.contrib.rnn.LSTMStateTuple(c=encoder_state_c, h=encoder_state_h)
    '''
    ################
    # Decoder by LSTM
    ################
    # Bahdanau attention
    def get_decoder_cell(rnn_size, num_layers, encoder_outputs, input_seq_len, slot_size):
        rnn_cell = get_lstm_cell(rnn_size)
        attention_mechanism = tf.contrib.seq2seq.BahdanauAttention(rnn_size, encoder_outputs,
                                                                   memory_sequence_length=input_seq_len)
        attn_cell = tf.contrib.seq2seq.DynamicAttentionWrapper(get_lstm_cell(rnn_size), 
                                                               attention_mechanism, 
                                                               attention_size=num_layers)
        #decoder_cell = tf.contrib.rnn.OutputProjectionWrapper(attn_cell, slot_size)
        return attn_cell
    
    # Struct LSTM cell in Decoder
    decoder_cell = tf.contrib.rnn.MultiRNNCell([get_decoder_cell(rnn_size, num_layers, encoder_outputs, input_seq_len, slot_size) for _ in range(num_layers)])
    
    # Output fully-connecting layer
    output_layer = Dense(vocab_size, 
                         kernel_initializer=tf.truncated_normal_initializer(mean = 0.0, stddev=0.1), 
                         kernel_regularizer=layers.l2_regularizer(0.1))
    
    # Struct training decoder
    with tf.variable_scope("decode"):
        training_helper = tf.contrib.seq2seq.TrainingHelper(decoder_embed_input,
                                                            target_seq_len)
        training_decoder = tf.contrib.seq2seq.BasicDecoder(decoder_cell, training_helper,
                                                           decoder_cell.zero_state(dtype=tf.float32, batch_size=batch_size), 
                                                           output_layer) 
        training_decoder_output, _ = tf.contrib.seq2seq.dynamic_decode(training_decoder,
                                                                       impute_finished=True,
                                                                       maximum_iterations=max_target_seq_len)
    # Struct predicting decoder (Share variable with training)
    with tf.variable_scope("decode", reuse=True):
        predicting_helper = tf.contrib.seq2seq.GreedyEmbeddingHelper(embeddings,
                                                                     start_tokens, 
                                                                     vocab[EOS_word])
        predicting_decoder = tf.contrib.seq2seq.BasicDecoder(decoder_cell, predicting_helper,
                                                             decoder_cell.zero_state(dtype=tf.float32, batch_size=batch_size), 
                                                             output_layer)
        predicting_decoder_output, _ = tf.contrib.seq2seq.dynamic_decode(predicting_decoder,
                                                                        impute_finished=True,
                                                                        maximum_iterations=limit_target_seq_len)
    
    ################
    # Model output tensor
    ################
    training_logits = tf.identity(training_decoder_output.rnn_output, name='trainings')
    predicting_logits = tf.identity(predicting_decoder_output.sample_id, name='predictions')
    
    ################
    # Optimization
    ################
    regularization_loss = tf.reduce_sum(tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES))
    with tf.name_scope("optimization"):
        masks = tf.sequence_mask(target_seq_len, max_target_seq_len, 
                                 dtype=tf.float32, name='masks')
        # Loss function
        cost = tf.contrib.seq2seq.sequence_loss(training_logits, targets, masks)
        # Optimizer
        optimizer = tf.train.AdamOptimizer(lr)
        # Gradient Clipping
        gradients = optimizer.compute_gradients(cost)
        train_step = optimizer.apply_gradients(gradients)
        #gradients, vriables = zip(*optimizer.compute_gradients(cost))
        #gradients, _ = tf.clip_by_global_norm(gradients, 5.0)
        #train_step = optimizer.apply_gradients(zip(gradients, vriables))

model_name = 'PTT_Encoder-Decoder_model_fixed_' + str(time.time())
mdl_dir = os.path.join(base_dir, 'model')
if not os.path.exists(mdl_dir):
    os.makedirs(mdl_dir)
model_dir = os.path.join(mdl_dir, model_name)
if not os.path.exists(model_dir):
    os.makedirs(model_dir)

num_instances = len(y_train)
iter_per_epoch = int(num_instances / batch_size)
iter_pct10 = int(iter_per_epoch / 5)
if (num_instances % batch_size) > 0:
    iter_per_epoch += 1
batch_cutoff = [0]
for i in range(iter_per_epoch - 1):
    batch_cutoff.append(batch_size * (i+1))
batch_cutoff.append(num_instances)

test_num_instances = len(y_test)
test_iter_per_epoch = int(test_num_instances / test_batch_size)

total_start_t = time.time()
best_validation_loss = 0.0
best_epoch = 1
early_stop_counter = 0

with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())
    
    for e in range(num_epoch):
        rand_idxs = np.random.permutation(num_instances)
        print('\n========')
        print('epoch' + str(e+1))
        print('========')
        start_t = time.time()
        
        #### Get random input batch and training model. ####
        for i in range(iter_per_epoch):
            X_batch = []
            Y_batch = []
            X_size = []
            Y_size = []
            for n in range(batch_cutoff[i],batch_cutoff[i+1]):
                X_batch.append(x_train[rand_idxs[n]])
                Y_batch.append(y_train[rand_idxs[n]])
                X_size.append(x_train_size[rand_idxs[n]])
                Y_size.append(y_train_size[rand_idxs[n]])
            max_X_size = max(X_size)
            max_Y_size = max(Y_size)
            X_batch = np.array(X_batch)
            Y_batch = np.array(Y_batch)
            X_size = np.array(X_size)
            Y_size = np.array(Y_size)
            
            _, loss = sess.run( [train_step, cost],
                                {inputs: X_batch[:,:max_X_size], targets: Y_batch[:,:max_Y_size],
                                 input_seq_len: X_size, target_seq_len: Y_size,
                                 limit_target_seq_len: limit_generation_len})
            
            if i % iter_pct10 == iter_pct10 - 1:
                print('Iteration ',i+1,' (',round(i*100/iter_per_epoch,2),'%) Train Loss: ',
                      loss,'\t\t',end='\n')
        
        print('Train Loss: ',loss)
        
        #### Get validation input batch and validating model. ####
        test_loss = []
        for i in range(test_iter_per_epoch):
            i_str = i * test_batch_size
            i_end = i_str + test_batch_size
            
            X_batch = np.array(x_test[i_str:i_end])
            Y_batch = np.array(y_test[i_str:i_end])
            X_size = np.array(x_test_size[i_str:i_end])
            Y_size = np.array(y_test_size[i_str:i_end])
            max_X_size = max(X_size)
            max_Y_size = max(Y_size)
            y_loss = sess.run( cost,
                                {inputs: X_batch[:,:max_X_size], targets: Y_batch[:,:max_Y_size],
                                 input_seq_len: X_size, target_seq_len: Y_size,
                                 limit_target_seq_len: limit_generation_len})
            test_loss.append(y_loss)
        else:
            test_loss = np.asarray(test_loss)
            validation_loss = test_loss.mean()
            validation_loss_std = test_loss.std()
        print('Test loss: ',validation_loss,'; Std: ',validation_loss_std)
        
        #### Generate title form testing content and validating model. ####
        answer_logits = sess.run( predicting_logits,
                                {inputs: X_batch[:,:max_X_size], targets: Y_batch[:,:max_Y_size], 
                                 input_seq_len: X_size, target_seq_len: Y_size,
                                 limit_target_seq_len: limit_generation_len})
        output_index = random.randint(0,test_batch_size-1)
        print('Target Words: {}'.format(" ".join([anti_vocab[i] for i in Y_batch[output_index] if i != 0])))
        print('Response Words: {}'.format(" ".join([anti_vocab[i] for i in answer_logits[output_index] if i != 0])))
        
        #### Save model weight parameters and information. ####
        if validation_loss < best_validation_loss or e == 0:
            best_validation_loss = validation_loss
            best_epoch = e
            early_stop_counter = 0
            print('Save best score!! '+str(best_validation_loss))
        else:
            early_stop_counter += 1
            print('Count early stop!! '+str(early_stop_counter))

        print('Elapsed time in epoch ' + str(e+1) + ': ' + str(time.time() - start_t) + ' [s]')

        model_path = os.path.join(model_dir, 'model-%d.h5' %(e+1))
        saver = tf.train.Saver()
        saver.save(sess, model_path)
        
        if patience != 0 and early_stop_counter >= patience:
            break

print('\n#######')
print('Best model')
print('#######')
print('Stop by early stopping')
print('Best score: ', best_validation_loss, 'Beat model: ', best_epoch)
print('Elapsed time in total: ' + str(time.time() - total_start_t))


epoch1
Iteration  216  ( 19.91 %) Train Loss:  6.15587 		
Iteration  432  ( 39.91 %) Train Loss:  5.59183 		
Iteration  648  ( 59.91 %) Train Loss:  4.804 		
Iteration  864  ( 79.91 %) Train Loss:  4.99025 		
Iteration  1080  ( 99.91 %) Train Loss:  4.4237 		
Train Loss:  4.4237
Test loss:  4.56659 ; Std:  0.141894
Target Words: <GO> 出身 好萊塢 女星 去世 享壽 91 <EOS>
Response Words: <GO> <GO> <GO> <GO> <GO> <GO> <GO> <GO> <GO> <GO> <GO> <GO> <GO> <GO> <GO> <GO> <GO> <GO> <GO> <GO> <GO> <GO>
Save best score!! 4.56659
Elapsed time in epoch 1: 768.9256250858307 [s]

epoch2
Iteration  216  ( 19.91 %) Train Loss:  4.38327 		
Iteration  432  ( 39.91 %) Train Loss:  4.31502 		
Iteration  648  ( 59.91 %) Train Loss:  4.22955 		
Iteration  864  ( 79.91 %) Train Loss:  3.76904 		
Iteration  1080  ( 99.91 %) Train Loss:  3.97885 		
Train Loss:  3.97885
Test loss:  4.1229 ; Std:  0.148089
Target Words: <GO> 1976 樂團 桃園 場 今晚 <EOS>
Response Words: <GO> 嗎 嗎 電影 電影 電影 電影 電影 電影 電影 電影 電影 電影 電影 電影 電影 電影 電影 電影 電影 電

Iteration  1080  ( 99.91 %) Train Loss:  3.06617 		
Train Loss:  3.06617
Test loss:  3.17591 ; Std:  0.185422
Target Words: <GO> 女生 根本 不用 追 <EOS>
Response Words: <GO> 陰道 級 級 級 級 級 級 級 級 級 級 級 級 級 級 級 級 級 級 級 級
Save best score!! 3.17591
Elapsed time in epoch 16: 756.9704468250275 [s]

epoch17
Iteration  216  ( 19.91 %) Train Loss:  2.81623 		
Iteration  432  ( 39.91 %) Train Loss:  3.00292 		
Iteration  648  ( 59.91 %) Train Loss:  2.82976 		
Iteration  864  ( 79.91 %) Train Loss:  2.83721 		
Iteration  1080  ( 99.91 %) Train Loss:  2.76791 		
Train Loss:  2.76791
Test loss:  3.16622 ; Std:  0.189698
Target Words: <GO> 西斯 大帝 都 怎麼 約 砲 <EOS>
Response Words: <GO> 拜託 傳產 傳產 傳產 播放 播放 播放 播放 播放 播放 播放 播放 播放 播放 播放 播放 播放 播放 播放 播放 播放
Save best score!! 3.16622
Elapsed time in epoch 17: 762.1114594936371 [s]

epoch18
Iteration  216  ( 19.91 %) Train Loss:  2.70734 		
Iteration  432  ( 39.91 %) Train Loss:  2.72897 		
Iteration  648  ( 59.91 %) Train Loss:  3.14687 		
Iteration  864  ( 79.91 %) Train 

In [53]:
import os
import random

base_dir = 'C:/Users/User/Raw data/PTT'
model_name = 'PTT_Encoder-Decoder_model_fixed_1522308771.903279'
epoch = 20
model_path = os.path.join(base_dir, 'model', model_name, 'model-{}.h5'.format(epoch))

loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # 加载模型
    loader = tf.train.import_meta_graph(model_path + '.meta')
    loader.restore(sess, model_path)

    inputs = loaded_graph.get_tensor_by_name('inputs:0')
    targets = loaded_graph.get_tensor_by_name('targets:0')
    logits = loaded_graph.get_tensor_by_name('predictions:0')
    input_seq_len = loaded_graph.get_tensor_by_name('input_seq_len:0')
    target_seq_len = loaded_graph.get_tensor_by_name('target_seq_len:0')
    limit_target_seq_len = loaded_graph.get_tensor_by_name('limit_target_seq_len:0')
    
    for _ in range(8):
        i_str = random.randint(0,test_num_instances-1)
        i_end = i_str + 1

        X_batch = np.tile(x_test[i_str], (test_batch_size, 1))
        Y_batch = np.tile(y_test[i_str], (test_batch_size, 1))
        X_size = np.tile(x_test_size[i_str], (test_batch_size))
        Y_size = np.tile(y_test_size[i_str], (test_batch_size))
        max_X_size = x_test_size[i_str]
        max_Y_size = y_test_size[i_str]
        answer_logits = sess.run( logits,
                                {inputs: X_batch, targets: Y_batch, 
                                 input_seq_len: X_size, target_seq_len: Y_size,
                                 limit_target_seq_len: limit_generation_len})

        output_index = 0 #random.randint(0,test_batch_size-1)
        print('\n========')
        print('Test index: ', i_str + output_index)
        print('========')
        print('Test input Words: {}'.format(" ".join([anti_vocab[i] for i in X_batch[output_index] if i != 0])))
        print('Target Words: {}'.format(" ".join([anti_vocab[i] for i in Y_batch[output_index] if i != 0])))
        print('Response Words: {}'.format(" ".join([anti_vocab[i] for i in answer_logits[output_index] if i != 0])))

INFO:tensorflow:Restoring parameters from C:/Users/User/Raw data/PTT\model\PTT_Encoder-Decoder_model_fixed_1522308771.903279\model-20.h5

Test index:  16996
Test input Words: <GO> 引述 einstean 台北 捷運 世界 一流 之銘言 88 話 看 完 的 感想 真的 是 很久沒 看過 劇情 這麼 神 的 漫畫 反觀 某個 伏筆 王 漫畫 借用 棒球 的 說法 畫工 是 可以 練 的 但是 創意 真的 是 天份 啊 88 話 揭露 艾倫 持有 的 進擊 的 巨人 是 為 了 追求 自由 而 奮戰 完美 解釋 了 前面 艾倫 變身 的 巨人 為何 突然 恢復 理智 舉起 了 堵住 被 打破 的 大門 初次 變身 巨人 後 艾倫 無法 掌控 巨人 的 力量 只能 憑著 本能 抹殺 其他 巨人 甚至 攻擊 米卡 莎 後來 因為 阿爾敏 的 提醒 艾倫 終於 想起 追求 自由 的 信念 而 恢復 理智 發揮 進擊 的 巨人 真正 的 力量 如果 作者 沒有 揭露 這個 設定 前面 這段 艾倫 恢復 理智 的 劇情 走向 就 會 有點 突兀 似乎 就是 單純 賣弄 熱血 而已 所以 作者 對於 進擊 的 巨人 設定 個人 覺得 還蠻 厲害 的 至於 有沒有 到 神 伏筆 那 就是 讀者 個人 觀感 了 對 我 來說 是 很 神 啦 <EOS>
Target Words: <GO> 進擊 的 巨人 那才 叫做 神 伏筆 <EOS>
Response Words: <GO> 南部 V V V V V V V V V V V V V V V V V V V V

Test index:  10862
Test input Words: <GO> 記得 很久以前 的 外國 電影 有 看過 一部 電影 很 好看 劇情 是 一 開始 四個 人 跑 去 搶銀行 事前 有 完整 的 規劃 警方 來 的 過程 中 還將 路面 炸毀 把 炸掉 下來 直接 開走 後來 搶到 錢 跑 去 類似 bar 包廂 慶祝 不過 後來 那些 搶匪 好像 一一 被 抓 了 記得 這部 電影 台灣 沒 上映 可是 真得 滿 好看 的 爬